In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import matplotlib.pyplot as plt

from helpers import show_model
from pomegranate import State, HiddenMarkovModel, DiscreteDistribution

In [6]:
model = HiddenMarkovModel(name="Example Model")

sunnyEmissions = DiscreteDistribution({"yes": 0.1, "no": 0.9})
sunnyState = State(sunnyEmissions, name="Sunny")

rainyEmissions = DiscreteDistribution({"yes": 0.8, "no": 0.2})
rainyState = State(rainyEmissions, name="Rainy")

model.add_states(sunnyState, rainyState)

assert rainyEmissions.probability("yes") == 0.8, "The director brings his umbrella with probability 0.8 on rainy days"

print("Looks good so far!")

Looks good so far!


In [12]:
model.add_transition(model.start, sunnyState, 0.5)
model.add_transition(model.start, rainyState, 0.5)

model.add_transition(sunnyState, sunnyState, 0.8)
model.add_transition(sunnyState, rainyState, 0.2)

model.add_transition(rainyState, rainyState, 0.6)
model.add_transition(rainyState, sunnyState, 0.4)

model.bake()

assert model.edge_count() == 6, "There should be two edges from model.start, two from Rainy, and two from Sunny"
assert model.node_count() == 4, "The states should include model.start, model.end, Rainy, and Sunny"

print("Great! You've finished the model.")

Great! You've finished the model.


In [15]:
show_model(model, figsize=(5, 5), filename="example.png", overwrite=True, show_ends=False)

ModuleNotFoundError: No module named 'pydotplus'

In [17]:
model.states

[{
     "class" : "State",
     "distribution" : {
         "class" : "Distribution",
         "dtype" : "str",
         "name" : "DiscreteDistribution",
         "parameters" : [
             {
                 "yes" : 0.8,
                 "no" : 0.2
             }
         ],
         "frozen" : false
     },
     "name" : "Rainy",
     "weight" : 1.0
 }, {
     "class" : "State",
     "distribution" : {
         "class" : "Distribution",
         "dtype" : "str",
         "name" : "DiscreteDistribution",
         "parameters" : [
             {
                 "yes" : 0.1,
                 "no" : 0.9
             }
         ],
         "frozen" : false
     },
     "name" : "Sunny",
     "weight" : 1.0
 }, {
     "class" : "State",
     "distribution" : null,
     "name" : "Example Model-start",
     "weight" : 1.0
 }, {
     "class" : "State",
     "distribution" : null,
     "name" : "Example Model-end",
     "weight" : 1.0
 }]

In [19]:
model.dense_transition_matrix()

array([[0.6, 0.4, 0. , 0. ],
       [0.2, 0.8, 0. , 0. ],
       [0.5, 0.5, 0. , 0. ],
       [0. , 0. , 0. , 0. ]])

In [23]:
column_order = ["Example Model-start", "Sunny", "Rainy", "Example Model-end"]
column_names = [s.name for s in model.states]
order_index = [column_names.index(c) for c in column_order]
transitions = model.dense_transition_matrix()[:, order_index][order_index, :]
transitions

array([[0. , 0.5, 0.5, 0. ],
       [0. , 0.8, 0.2, 0. ],
       [0. , 0.4, 0.6, 0. ],
       [0. , 0. , 0. , 0. ]])

In [25]:
observations = ['yes', 'no', 'yes']

assert len(observations) > 0, "You need to choose a sequence of 'yes'/'no' observations to test"

forward_matrix = np.exp(model.forward(observations))
probability_percentage = np.exp(model.log_probability(observations))
probability_percentage

0.06920000000000004

In [27]:
observations = ['yes', 'no', 'yes']

viterbi_likelihood, viterbi_path = model.viterbi(observations)
print(np.exp(viterbi_likelihood))
print(viterbi_path)

0.023040000000000005
[(2, {
    "class" : "State",
    "distribution" : null,
    "name" : "Example Model-start",
    "weight" : 1.0
}), (0, {
    "class" : "State",
    "distribution" : {
        "class" : "Distribution",
        "dtype" : "str",
        "name" : "DiscreteDistribution",
        "parameters" : [
            {
                "yes" : 0.8,
                "no" : 0.2
            }
        ],
        "frozen" : false
    },
    "name" : "Rainy",
    "weight" : 1.0
}), (1, {
    "class" : "State",
    "distribution" : {
        "class" : "Distribution",
        "dtype" : "str",
        "name" : "DiscreteDistribution",
        "parameters" : [
            {
                "yes" : 0.1,
                "no" : 0.9
            }
        ],
        "frozen" : false
    },
    "name" : "Sunny",
    "weight" : 1.0
}), (0, {
    "class" : "State",
    "distribution" : {
        "class" : "Distribution",
        "dtype" : "str",
        "name" : "DiscreteDistribution",
        "para